# How to use the pre-trained model in your own code

### 1. Complete the Setup instructions from the Readme and download the pre-trained models
- The models were tested with Kears v. 2.1.5 and tensorflow 1.6.0

In [1]:
import sys

In [2]:
# Make sure that the directory of the project is in your Python PATH
sys.path.insert(0, "relation_extraction/")

In [ ]:
from core.parser import RelParser
from core import keras_models

### 2. You need to tokenize and part-of-speech tag your data
- The easiest way to so is to use Stanford CoreNLP server with the pycorenlp library
- Install and start the CoreNLP server with english models as instructed here: [CoreNLP Server](https://stanfordnlp.github.io/CoreNLP/corenlp-server.html)
- Install the pycorenlp python library: `pip install pycorenlp`

In [4]:
from pycorenlp import StanfordCoreNLP

In [5]:
corenlp = StanfordCoreNLP('http://localhost:9000')
corenlp_properties = {
    'annotators': 'tokenize, pos, ner',
    'outputFormat': 'json'
}

In [6]:
def get_tagged_from_server(input_text):
    """
    Send the input_text to the CoreNLP server and retrieve the tokens, named entity tags and part-of-speech tags.
    """
    corenlp_output = corenlp.annotate(input_text,properties=corenlp_properties).get("sentences", [])[0]
    tagged = [(t['originalText'], t['ner'], t['pos']) for t in corenlp_output['tokens']]
    return tagged

In [7]:
print(get_tagged_from_server("Germany is a country in Europe"))

[('Germany', 'LOCATION', 'NNP'), ('is', 'O', 'VBZ'), ('a', 'O', 'DT'), ('country', 'O', 'NN'), ('in', 'O', 'IN'), ('Europe', 'LOCATION', 'NNP')]


In [8]:
print(get_tagged_from_server("Star Wars VII is an American space opera epic film directed by  J. J. Abrams."))

[('Star', 'O', 'NNP'), ('Wars', 'O', 'NNP'), ('VII', 'O', 'NNP'), ('is', 'O', 'VBZ'), ('an', 'O', 'DT'), ('American', 'MISC', 'JJ'), ('space', 'O', 'NN'), ('opera', 'O', 'NN'), ('epic', 'O', 'NN'), ('film', 'O', 'NN'), ('directed', 'O', 'VBN'), ('by', 'O', 'IN'), ('J.', 'PERSON', 'NNP'), ('J.', 'PERSON', 'NNP'), ('Abrams', 'PERSON', 'NNP'), ('.', 'O', '.')]


- You can also generate a similar output with any part-of-speech tagger of your choice und use it with our models.

### 3. Extract entity mentions and generate an empty graph of relations in the input sentence

In [9]:
from core import entity_extraction

In [10]:
# Convert the input string into a list of tuples with the Stanford CoreNLP as explained above
tagged = get_tagged_from_server("Germany is a country in Europe")

In [12]:
entity_fragments = entity_extraction.extract_entities(tagged)
edges = entity_extraction.generate_edges(entity_fragments)
non_parsed_graph = {'tokens': [t for t, _, _ in tagged],
                    'edgeSet': edges}
print(edges)

[{'left': [0], 'right': [5]}, {'left': [0], 'right': [3]}, {'left': [5], 'right': [3]}]


- Empty relations are called edges and they have two attributes: 'left' and 'right' that contain token indices of entity mentions

### 4. Load the pre-trained relation extraction model

In [13]:
# the glove embeddings should be in the "resources/" folder, otherwise change the pathes in the model_params.json or directly in the code
keras_models.model_params['wordembeddings'] = "../resources/embeddings/glove/glove.6B.50d.txt"

In [15]:
# The downloaded pretrained models should be in the "trainedmodels/" folder
relparser = RelParser("model_ContextWeighted", models_foldes="trainedmodels/")

Loaded embeddings: (400002, 50)
Parameters: {'rnn1_layers': 1, 'bidirectional': False, 'units1': 256, 'dropout1': 0.5, 'optimizer': 'adam', 'window_size': 3, 'position_emb': 3, 'batch_size': 128, 'gpu': True, 'property2idx': 'property2idx.txt', 'wordembeddings': '../resources/embeddings/glove/glove.6B.50d.txt', 'max_sent_len': 36}


### 5. Label the edges in the sentence graph using the pre-trained model

In [16]:
parsed_graph = relparser.classify_graph_relations(non_parsed_graph)

100%|##########| 1/1 [00:00<00:00, 5197.40it/s]


- The output is a dictionary, the labeled edges are stored in the 'edgeSet' field. 
- 'kbID' contains the wikidata identifier of the assigned relation, 'P0' stands for an empty relation
- 'lexicalInput' contains a human readable relation label

In [17]:
print(parsed_graph)

{'tokens': ['Germany', 'is', 'a', 'country', 'in', 'Europe'], 'edgeSet': [{'left': [0], 'right': [5], 'kbID': 'P30', 'lexicalInput': 'continent'}, {'left': [0], 'right': [3], 'kbID': 'P0', 'lexicalInput': 'ALL_ZERO'}, {'left': [5], 'right': [3], 'kbID': 'P31', 'lexicalInput': 'instance of'}]}
